## Scraping Aged Care Facilties Reviews

In [1]:
# import
import requests
from bs4 import BeautifulSoup
import urllib.request, urllib.parse, urllib.error
from urllib.parse import urljoin
import lxml
import html
import re
from selenium import webdriver
import pandas as pd

In [2]:
import sqlite3
sqlite_db = './openlib.db'
conn = sqlite3.connect(sqlite_db) 
c = conn.cursor()

In [3]:
c = sqlite3.connect('./openlib.db')

def Q(query, db=c):
    return sql.read_sql(query, db)

In [4]:
# import sleep
from time import sleep

In [16]:
# Backup
# visit our relevant page
driver = webdriver.Chrome(executable_path="./chromedriver/chromedriver.exe")
driver.get("https://agedcarereviews.com.au/searchresult.php?search=&suburb=Hunters+Hill+-+NSW&lat=&lng=&range=30")
# wait one second
sleep(1)
#grab the page source
html = driver.page_source

In [17]:
# BeautifulSoup it!
html = BeautifulSoup(html,'lxml')

In [18]:
len(html)

7

In [19]:
import pandas as pd
box = pd.DataFrame(columns=["f_link","name","rate_v","text","qty","review"])

In [20]:
for entry in html.find_all('div',"box"):

    try:
       # grab the link
        for link in entry.find_all('div',"upper"):
            a = link.find('a')
            f_link = (a.attrs['href'])
            #print (f_link)


        # grab name and the rating       
        driver.get(f_link)
        sleep(1)
        s_html=driver.page_source
        s_html= BeautifulSoup(s_html, 'lxml')
        try:
            for s2 in s_html.find_all('div',"left-side-title"):
                name=(s2.find('h1', {'itemprop': 'name'}).text)
                rate_v=(s2.find('span', {'itemprop': 'ratingValue'}).text)
        except:
            entry='no records'            

        try:
            for s3 in s_html.find_all('div',"review_small_right"):
                text=(s3.find('div', {'class': 'tab_review_bottom active'}).text)
                for s4 in s3.find_all('div',"tab_review_bottom active"):
                    qty=(s4.find('div', {'class': 'star_rating'}).attrs)
        except:
            entry='no records'   
        
        try:
            for s5 in s_html.find_all('div',"comment_container"):
                for s6 in s5.find_all('div',"right_comment"):
                    review=(s6.find('div', {'class': 'inner-comment-container'}).text)
        except:
            entry='no records'       
    except:
        entry='no records'

    box.loc[len(box)]=[f_link,name,rate_v,text,qty,review]

In [21]:
box.to_sql(name = 'box', con = conn, if_exists = 'replace', index = False)

In [22]:
box.to_csv('C:/Users/lynn_/OneDrive/Documents/Lynn/general_assembly/box.csv')

In [89]:
box.name.value_counts()

UnitingCare The Marion                                                      1
Presbyterian Aged Care Ashfield                                             1
Ashburn House Aged Care Facility                                            1
IRT Woolwich                                                                1
UnitingCare Community Care East                                             1
Allied Healthcare Australia North Ryde                                      1
UnitingCare St Columba's Retirement Centre                                  1
Marian Hostel                                                               1
James Milson Village                                                        1
RFBI Concord Community Hostel                                               1
Russell Lea Nursing Home                                                    1
St Mary's Villa Concord                                                     1
Clermont Aged Care                                              

In [82]:
# close it
driver.close()

## Notes

In [ ]:
test_str = '''
<div class="star_rating" data-score="4.34" style="float: 
left; width: 100px;" title="very good" id="ui-id-1">
<img src="https://agedcarereviews.com.au/js/raty/img/star-on.png" alt="1" title="very good">&nbsp;
<img src="https://agedcarereviews.com.au/js/raty/img/star-on.png" alt="2" title="very good">&nbsp;
<img src="https://agedcarereviews.com.au/js/raty/img/star-on.png" alt="3" title="very good">&nbsp;
<img src="https://agedcarereviews.com.au/js/raty/img/star-on.png" alt="4" title="very good">&nbsp;
<img src="https://agedcarereviews.com.au/js/raty/img/star-half.png" alt="5" title="very good">
<input type="hidden" name="score" value="4.34" readonly="readonly"></div>

'''

In [ ]:
test_str

In [ ]:
test_str_soup = BeautifulSoup(test_str,'lxml')

In [ ]:
found_a = test_str_soup.find('div')
found_link = found_a.attrs['data-score']
print (found_link)

In [ ]:
base_url='https://au.indeed.com/'


for entry in html.find_all('div',"row result clickcard"):

    try:
       # grab the link
        for link in entry.find_all('h2', {'class': 'jobtitle'}):
            a = link.find('a')
            f_link = urljoin(base_url,a.attrs['href'])
            #print (f_link)


        # grab full job summary        
        driver.get(f_link)
        sleep(1)
        s_html=driver.page_source
        s_html= BeautifulSoup(s_html, 'lxml')
        for s2 in s_html.find_all('table', {'id': 'job-content'}):
            s1=(s2.find('span', {'class': 'summary'}).text)
            #print(s1)

        # grab the title
        for t in entry.findAll('h2', {'class': 'jobtitle'}):
            b = t.find('a')
            title=(b.text.strip())

        # grab the company
        try:
            company = (entry.find('span', "company").text)
            if company:
                company = (entry.find('span', "company").text)
        except:
            company='NA'

        # grab the location
        try:
            location=(entry.find('span', {'class': 'location'}).text)
            if location:
                location = (entry.find('span', {'class': 'location'}).text)
        except:
            location='NA'

        # grab the salary scale
        try:
            salaryscale=(entry.find('span', class_="no-wrap").text)
            if salaryscale:
                salaryscale=(entry.find('span', class_="no-wrap").text)
        except:
            salaryscale='NA'

        # grab the summary
        try:
            summary = (entry.find('span', {'class': 'summary'}).text)
            if summary:
                summary = (entry.find('span', {'class': 'summary'}).text)
        except:
            summary='NA'

        # grab the job sponsor
        try:
            sponsor = (entry.find('span', {'class': 'sponsoredGray'}).text)
            if sponsor:
                sponsor = (entry.find('span', {'class': 'sponsoredGray'}).text)
        except:
            sponsor='NA'

        # grab the date
        try:
            date = (entry.find('span', class_="date").text)
            if date:
                date = (entry.find('span', class_="date").text)
        except:
            date='NA'

    except:
        entry='no records'

    i_card.loc[len(i_card)]=[f_link,title,company,location,salaryscale,summary,sponsor,date,s1]
